## Import

In [ ]:
import os
# Change to available GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

In [ ]:
import nltk
import numpy as np
import sklearn_crfsuite
import tensorflow as tf
import torch
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from seqeval.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn_crfsuite import metrics
from keras.models import Model
from keras.layers import Input, LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from torch import cuda
from torch.utils.data import DataLoader, Dataset
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    BertConfig,
    BertForTokenClassification,
    BertTokenizerFast,
)
from tqdm import tqdm
from utils.bert_utils import dataset
from utils.file_utils import open_conll_file

## Prepare File

In [ ]:
# File format must be conll or conllu
train_data_path = ""
val_data_path = ""
test_data_path = ""

# Read POS tagged data
train_data = open_conll_file(train_data_path)
val_data = open_conll_file(val_data_path)
test_data = open_conll_file(test_data_path)

## Model

### BERT

In [ ]:
base_model = "model/indobert-large-p2-finetuned-pos"
tokenizer = AutoTokenizer.from_pretrained(base_model)

In [ ]:
train_sentences = [[word[0] for word in sentence] for sentence in train_data]
val_sentences = [[word[0] for word in sentence] for sentence in val_data]
test_sentences = [[word[0] for word in sentence] for sentence in test_data]

In [ ]:
train_labels = [[s[1] for s in sentence] for sentence in train_data]
val_labels = [[s[1] for s in sentence] for sentence in val_data]
test_labels = [[s[1] for s in sentence] for sentence in test_data]

In [ ]:
unique_labels = list(
    set([lab for label in train_labels + val_labels + test_labels for lab in label])
)
labels_to_ids = {k: v for v, k in enumerate(sorted(unique_labels))}
ids_to_labels = {v: k for v, k in enumerate(sorted(unique_labels))}

In [ ]:
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
## Parameters
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 50
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10

In [ ]:
training_set = dataset(train_sentences, train_labels, tokenizer, MAX_LEN, labels_to_ids)
validation_set = dataset(val_sentences, val_labels, tokenizer, MAX_LEN, labels_to_ids)
testing_set = dataset(test_sentences, test_labels, tokenizer, MAX_LEN, labels_to_ids)

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(validation_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
model = BertForTokenClassification.from_pretrained(base_model, num_labels=len(unique_labels), id2label=ids_to_labels, label2id=labels_to_ids)
model.to(device)

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [ ]:
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    val_loss, val_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    nb_val_examples, nb_val_steps = 0, 0
    tr_preds, tr_labels = [], []
    val_preds, val_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(tqdm(training_loader)):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss = outputs[0]
        tr_logits = outputs[1]
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        # if idx % 100==0:
        #     loss_step = tr_loss/nb_tr_steps
        #     print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_step = tr_loss/nb_tr_steps
    
    model.eval()
    for idx, batch in enumerate(val_loader):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss = outputs[0]
        val_logits = outputs[1]
        val_loss += loss.item()

        nb_val_steps += 1
        nb_val_examples += labels.size(0)
        
        # if idx % 100==0:
        #     loss_step = val_loss/nb_val_steps
        #     print(f"Validation loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = val_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        val_labels.extend(labels)
        val_preds.extend(predictions)

        tmp_val_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        val_accuracy += tmp_val_accuracy
    

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")
    print(f"Validation loss epoch: {val_loss/nb_val_steps}")
    print(f"Validation accuracy epoch: {val_accuracy/nb_val_steps}")

In [ ]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

In [ ]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    # with torch.no_grad():
    for idx, batch in enumerate(testing_loader):

        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss = outputs[0]
        eval_logits = outputs[1]

        eval_loss += loss.item()

        nb_eval_steps += 1
        nb_eval_examples += labels.size(0)

        # if idx % 100==0:
        #     loss_step = eval_loss/nb_eval_steps
        #     print(f"Validation loss per 100 evaluation steps: {loss_step}")

        # compute evaluation accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)

        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)

        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        eval_labels.extend(labels)
        eval_preds.extend(predictions)

        tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        eval_accuracy += tmp_eval_accuracy

    labels = [[model.config.id2label[id.item()]] for id in eval_labels]
    predictions = [[model.config.id2label[id.item()]] for id in eval_preds]
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [ ]:
labels, predictions = valid(model, testing_loader)

In [ ]:
print(classification_report(labels, predictions, digits=4))
print(metrics.flat_classification_report(labels, predictions, digits=4))